In [ ]:
!python --version

Python 3.10.12


In [ ]:
!sudo apt-get install build-essential
!sudo apt-get install libatlas-base-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# install python 3.8
!sudo apt-get update -y
!sudo apt-get install python3.8

# change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2

# check python version
!python --version

# install pip for new python
!sudo apt-get install python3.8-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

!python -m pip install pip==24.0

# credit of these last two commands blongs to @Erik
# install colab's dependencies
!python -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.10/dist-packages/google \
       /usr/local/lib/python3.8/dist-packages/google

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,323 kB]
Get:13 http://archive.ubuntu.c

In [ ]:
!python --version

Python 3.8.20


In [ ]:
!nvidia-smi

Sun Nov 24 21:40:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!git clone https://github.com/AlesSrsen/spacetimeformer.git

Cloning into 'spacetimeformer'...
remote: Enumerating objects: 636, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 636 (delta 199), reused 192 (delta 178), pack-reused 369 (from 1)
Receiving objects: 100% (636/636), 16.23 MiB | 10.77 MiB/s, done.
Resolving deltas: 100% (338/338), done.


In [ ]:
!cd /content/spacetimeformer

In [ ]:
# DO NOT RESTART THE RUNTIME - IT BREAKS THE PYTHON 3.8 The runtime may not even start
!pip install -r requirements.txt
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 69.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.1/582.1 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 10.4 MB/s eta 0:00:

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.2 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.6.0 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!pip install .
# If editable then
# !pip install -e .

Processing /content/spacetimeformer
  Preparing metadata (setup.py) ... done
  Created wheel for spacetimeformer: filename=spacetimeformer-1.5.0-py3-none-any.whl size=95648 sha256=bfcf2c2ba4bef02e2cd3aedad9b368a14b19f02e30551737e0e9dd87c764d082
  Stored in directory: /tmp/pip-ephem-wheel-cache-j2c7vy8b/wheels/ac/88/d0/6707b49e2b1821c10fef5d990137d22b88c9a7f5eb1b288833
Successfully built spacetimeformer
DEPRECATION: pytorch-lightning 1.6.0 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content/spacetimeformer


# Pull and update content from the repository

In [ ]:
!cd /content/spacetimeformer/ && git pull origin main
!cd /content/spacetimeformer && git checkout main -- spacetimeformer/train.py

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 4), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 501 bytes | 501.00 KiB/s, done.
From https://github.com/AlesSrsen/spacetimeformer
 * branch            main       -> FETCH_HEAD
   4f891cb..4aa77fc  main       -> origin/main
Updating 4f891cb..4aa77fc
Fast-forward
 spacetimeformer/time2vec.py | 2 +-
 spacetimeformer/train.py    | 2 +-
 2 files changed, 2 insertions(+), 2 deletions(-)


In [ ]:
#!cat /content/spacetimeformer/spacetimeformer/train.py

In [ ]:
!ls /content/drive/MyDrive/ColabNotebooks/DataminingAndTimeSeries

crypto_preparation.ipynb				spacetimeformer_google_colab.ipynb
preprocessed_12_cols_500_rows_eth_timeseries.csv	wandb
preprocessed_12_cols_500_rows_noNaN_eth_timeseries.csv


In [ ]:
%env STF_WANDB_ACCT=johannes_thyroff
%env STF_WANDB_PROJ=DataminingAndTimeseries
%env STF_LOG_DIR=/content/drive/MyDrive/ColabNotebooks/DataminingAndTimeSeries/wandb/

env: STF_WANDB_ACCT=johannes_thyroff
env: STF_WANDB_PROJ=DataminingAndTimeseries
env: STF_LOG_DIR=/content/drive/MyDrive/ColabNotebooks/DataminingAndTimeSeries/wandb/


In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# use --gpus 0
# https://github.com/QData/spacetimeformer/issues/90#issuecomment-2220829221
# Github Save Change

#--time_emb_dim

# TODO
!echo $STF_WANDB_ACCT
!echo $STF_WANDB_PROJ
print('Username and projectname have to be without " . ')
#
!WANDB_API_KEY=HERE_YOUR_WANDB_API_KEY_DONT_PUSH_TO_GITHUB; python spacetimeformer/train.py spacetimeformer dmts_crypto --data_path /content/drive/MyDrive/ColabNotebooks/DataminingAndTimeSeries/preprocessed_12_cols_500_rows_noNaN_eth_timeseries.csv --wandb --batch_size 24 --embed_method spatio-temporal --local_self_attn full --local_cross_attn full --global_self_attn full --global_cross_attn full --run_name dmts_first_run_stf --context_points 10  --gpus 0


johannes_thyroff
DataminingAndTimeseries
Username and projectname have to be without " . 
wandb: Currently logged in as: johannes_thyroff. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /content/drive/MyDrive/ColabNotebooks/DataminingAndTimeSeries/wandb/wandb/run-20241124_225457-73celrfs
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run gallant-galaxy-5
wandb: ⭐️ View project at https://wandb.ai/johannes_thyroff/DataminingAndTimeseries
wandb: 🚀 View run at https://wandb.ai/johannes_thyroff/DataminingAndTimeseries/runs/73celrfs
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/lightning.py:2046: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_

In [ ]:
from argparse import ArgumentParser
import random
import sys
import warnings
import os
import uuid

import pytorch_lightning as pl
import torch

import spacetimeformer as stf

_MODELS = ["spacetimeformer", "mtgnn", "heuristic", "lstm", "lstnet", "linear", "s4"]

_DSETS = [
    "dmts_crypto" # DMTS Modification
]


def create_model(config):
    x_dim, yc_dim, yt_dim = None, None, None
    if config.dset == "dmts_crypto": # DMTS Modification
        x_dim = 12
        yc_dim = 1
        yt_dim = 1
    assert x_dim is not None
    assert yc_dim is not None
    assert yt_dim is not None

    if config.model == "lstm":
        forecaster = stf.lstm_model.LSTM_Forecaster(
            # encoder
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            time_emb_dim=config.time_emb_dim,
            hidden_dim=config.hidden_dim,
            n_layers=config.n_layers,
            dropout_p=config.dropout_p,
            # training
            learning_rate=config.learning_rate,
            teacher_forcing_prob=config.teacher_forcing_start,
            l2_coeff=config.l2_coeff,
            loss=config.loss,
            linear_window=config.linear_window,
            use_revin=config.use_revin,
            linear_shared_weights=config.linear_shared_weights,
            use_seasonal_decomp=config.use_seasonal_decomp,
        )

    elif config.model == "heuristic":
        forecaster = stf.heuristic_model.Heuristic_Forecaster(
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            context_points=config.context_points,
            target_points=config.target_points,
            loss=config.loss,
            method=config.method,
        )
    elif config.model == "mtgnn":
        forecaster = stf.mtgnn_model.MTGNN_Forecaster(
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            context_points=config.context_points,
            target_points=config.target_points,
            gcn_depth=config.gcn_depth,
            dropout_p=config.dropout_p,
            node_dim=config.node_dim,
            dilation_exponential=config.dilation_exponential,
            conv_channels=config.conv_channels,
            subgraph_size=config.subgraph_size,
            skip_channels=config.skip_channels,
            end_channels=config.end_channels,
            residual_channels=config.residual_channels,
            layers=config.layers,
            propalpha=config.propalpha,
            tanhalpha=config.tanhalpha,
            learning_rate=config.learning_rate,
            kernel_size=config.kernel_size,
            l2_coeff=config.l2_coeff,
            time_emb_dim=config.time_emb_dim,
            loss=config.loss,
            linear_window=config.linear_window,
            linear_shared_weights=config.linear_shared_weights,
            use_seasonal_decomp=config.use_seasonal_decomp,
            use_revin=config.use_revin,
        )
    elif config.model == "lstnet":
        forecaster = stf.lstnet_model.LSTNet_Forecaster(
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            context_points=config.context_points,
            hidRNN=config.hidRNN,
            hidCNN=config.hidCNN,
            hidSkip=config.hidSkip,
            CNN_kernel=config.CNN_kernel,
            skip=config.skip,
            dropout_p=config.dropout_p,
            output_fun=config.output_fun,
            learning_rate=config.learning_rate,
            l2_coeff=config.l2_coeff,
            loss=config.loss,
            linear_window=config.linear_window,
            use_revin=config.use_revin,
        )
    elif config.model == "spacetimeformer":
        if hasattr(config, "context_points") and hasattr(config, "target_points"):
            max_seq_len = config.context_points + config.target_points
        elif hasattr(config, "max_len"):
            max_seq_len = config.max_len
        else:
            raise ValueError("Undefined max_seq_len")
        forecaster = stf.spacetimeformer_model.Spacetimeformer_Forecaster(
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            max_seq_len=max_seq_len,
            start_token_len=config.start_token_len,
            attn_factor=config.attn_factor,
            d_model=config.d_model,
            d_queries_keys=config.d_qk,
            d_values=config.d_v,
            n_heads=config.n_heads,
            e_layers=config.enc_layers,
            d_layers=config.dec_layers,
            d_ff=config.d_ff,
            dropout_emb=config.dropout_emb,
            dropout_attn_out=config.dropout_attn_out,
            dropout_attn_matrix=config.dropout_attn_matrix,
            dropout_qkv=config.dropout_qkv,
            dropout_ff=config.dropout_ff,
            pos_emb_type=config.pos_emb_type,
            use_final_norm=not config.no_final_norm,
            global_self_attn=config.global_self_attn,
            local_self_attn=config.local_self_attn,
            global_cross_attn=config.global_cross_attn,
            local_cross_attn=config.local_cross_attn,
            performer_kernel=config.performer_kernel,
            performer_redraw_interval=config.performer_redraw_interval,
            attn_time_windows=config.attn_time_windows,
            use_shifted_time_windows=config.use_shifted_time_windows,
            norm=config.norm,
            activation=config.activation,
            init_lr=config.init_lr,
            base_lr=config.base_lr,
            warmup_steps=config.warmup_steps,
            decay_factor=config.decay_factor,
            initial_downsample_convs=config.initial_downsample_convs,
            intermediate_downsample_convs=config.intermediate_downsample_convs,
            embed_method=config.embed_method,
            l2_coeff=config.l2_coeff,
            loss=config.loss,
            class_loss_imp=config.class_loss_imp,
            recon_loss_imp=config.recon_loss_imp,
            time_emb_dim=config.time_emb_dim,
            null_value=config.null_value,
            pad_value=config.pad_value,
            linear_window=config.linear_window,
            use_revin=config.use_revin,
            linear_shared_weights=config.linear_shared_weights,
            use_seasonal_decomp=config.use_seasonal_decomp,
            use_val=not config.no_val,
            use_time=not config.no_time,
            use_space=not config.no_space,
            use_given=not config.no_given,
            recon_mask_skip_all=config.recon_mask_skip_all,
            recon_mask_max_seq_len=config.recon_mask_max_seq_len,
            recon_mask_drop_seq=config.recon_mask_drop_seq,
            recon_mask_drop_standard=config.recon_mask_drop_standard,
            recon_mask_drop_full=config.recon_mask_drop_full,
        )
    elif config.model == "linear":
        forecaster = stf.linear_model.Linear_Forecaster(
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            context_points=config.context_points,
            learning_rate=config.learning_rate,
            l2_coeff=config.l2_coeff,
            loss=config.loss,
            linear_window=config.linear_window,
            linear_shared_weights=config.linear_shared_weights,
            use_revin=config.use_revin,
            use_seasonal_decomp=config.use_seasonal_decomp,
        )
    elif config.model == "s4":
        forecaster = stf.s4_model.S4_Forecaster(
            context_points=config.context_points,
            target_points=config.target_points,
            d_state=config.d_state,
            d_model=config.d_model,
            d_x=x_dim,
            d_yc=yc_dim,
            d_yt=yt_dim,
            layers=config.layers,
            time_emb_dim=config.time_emb_dim,
            channels=config.channels,
            dropout_p=config.dropout_p,
            learning_rate=config.learning_rate,
            l2_coeff=config.l2_coeff,
            loss=config.loss,
            linear_window=config.linear_window,
            linear_shared_weights=config.linear_shared_weights,
            use_revin=config.use_revin,
            use_seasonal_decomp=config.use_seasonal_decomp,
        )

    return forecaster


def create_dset(config):
    INV_SCALER = lambda x: x
    SCALER = lambda x: x
    NULL_VAL = None
    PLOT_VAR_IDXS = None
    PLOT_VAR_NAMES = None
    PAD_VAL = None

    time_col_name = "Datetime"
    data_path = config.data_path
    time_features = ["year", "month", "day", "weekday", "hour", "minute"]

    if config.dset == "dmts_crypto": # DMTS Modification
        if data_path == "auto":
            raise ValueError("Please specify a datapath.")
        target_cols = ["close"]
        time_col_name = "time"

    dset = stf.data.CSVTimeSeries(
        data_path=data_path,
        target_cols=target_cols,
        ignore_cols="all",
        time_col_name=time_col_name,
        time_features=time_features,
        val_split=0.2,
        test_split=0.2,
    )
    DATA_MODULE = stf.data.DataModule(
        datasetCls=stf.data.CSVTorchDset,
        dataset_kwargs={
            "csv_time_series": dset,
            "context_points": config.context_points,
            "target_points": config.target_points,
            "time_resolution": config.time_resolution,
        },
        batch_size=config.batch_size,
        workers=config.workers,
        overfit=args.overfit,
    )
    INV_SCALER = dset.reverse_scaling
    SCALER = dset.apply_scaling
    NULL_VAL = None

    return (
        DATA_MODULE,
        INV_SCALER,
        SCALER,
        NULL_VAL,
        PLOT_VAR_IDXS,
        PLOT_VAR_NAMES,
        PAD_VAL,
    )


def create_callbacks(config, save_dir):
    filename = f"{config.run_name}_" + str(uuid.uuid1()).split("-")[0]
    model_ckpt_dir = os.path.join(save_dir, filename)
    config.model_ckpt_dir = model_ckpt_dir
    saving = pl.callbacks.ModelCheckpoint(
        dirpath=model_ckpt_dir,
        monitor="val/loss",
        mode="min",
        filename=f"{config.run_name}" + "{epoch:02d}",
        save_top_k=1,
        auto_insert_metric_name=True,
    )
    callbacks = [saving]

    if not config.no_earlystopping:
        callbacks.append(
            pl.callbacks.early_stopping.EarlyStopping(
                monitor="val/loss",
                patience=config.patience,
            )
        )

    if config.wandb:
        callbacks.append(pl.callbacks.LearningRateMonitor())

    if config.model == "lstm":
        callbacks.append(
            stf.callbacks.TeacherForcingAnnealCallback(
                start=config.teacher_forcing_start,
                end=config.teacher_forcing_end,
                steps=config.teacher_forcing_anneal_steps,
            )
        )
    if config.time_mask_loss:
        callbacks.append(
            stf.callbacks.TimeMaskedLossCallback(
                start=config.time_mask_start,
                end=config.time_mask_end,
                steps=config.time_mask_anneal_steps,
            )
        )
    return callbacks



In [ ]:
# def main(args)

log_dir = os.getenv("STF_LOG_DIR")
if log_dir is None:
    log_dir = "./data/STF_LOG_DIR"
    print(
        "Using default wandb log dir path of ./data/STF_LOG_DIR. This can be adjusted with the environment variable `STF_LOG_DIR`"
    )
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

if args.wandb:
    import wandb

    project = os.getenv("STF_WANDB_PROJ")
    entity = os.getenv("STF_WANDB_ACCT")
    assert (
        project is not None and entity is not None
    ), "Please set environment variables `STF_WANDB_ACCT` and `STF_WANDB_PROJ` with \n\
        your wandb user/organization name and project title, respectively."
    experiment = wandb.init(
        project=project,
        entity=entity,
        config=args,
        dir=log_dir,
        reinit=True,
    )
    config = wandb.config
    wandb.run.name = args.run_name
    wandb.run.save()
    logger = pl.loggers.WandbLogger(
        experiment=experiment,
        save_dir=log_dir,
    )

# Dset
(
    data_module,
    inv_scaler,
    scaler,
    null_val,
    plot_var_idxs,
    plot_var_names,
    pad_val,
) = create_dset(args)

# Model
args.null_value = null_val
args.pad_value = pad_val
forecaster = create_model(args)
forecaster.set_inv_scaler(inv_scaler)
forecaster.set_scaler(scaler)
forecaster.set_null_value(null_val)

# Callbacks
callbacks = create_callbacks(args, save_dir=log_dir)
test_samples = next(iter(data_module.test_dataloader()))

if args.wandb and args.plot:
    callbacks.append(
        stf.plot.PredictionPlotterCallback(
            test_samples,
            var_idxs=plot_var_idxs,
            var_names=plot_var_names,
            pad_val=pad_val,
            total_samples=min(args.plot_samples, args.batch_size),
        )
    )

if args.wandb and args.model == "spacetimeformer" and args.attn_plot:

    callbacks.append(
        stf.plot.AttentionMatrixCallback(
            test_samples,
            layer=0,
            total_samples=min(16, args.batch_size),
        )
    )

if args.wandb:
    config.update(args)
    logger.log_hyperparams(config)

if args.val_check_interval <= 1.0:
    val_control = {"val_check_interval": args.val_check_interval}
else:
    val_control = {"check_val_every_n_epoch": int(args.val_check_interval)}

trainer = pl.Trainer(
    gpus=args.gpus,
    callbacks=callbacks,
    logger=logger if args.wandb else None,
    accelerator="dp",
    gradient_clip_val=args.grad_clip_norm,
    gradient_clip_algorithm="norm",
    overfit_batches=20 if args.debug else 0,
    accumulate_grad_batches=args.accumulate,
    sync_batchnorm=True,
    limit_val_batches=args.limit_val_batches,
    **val_control,
)

# Train
trainer.fit(forecaster, datamodule=data_module)

# Test
trainer.test(datamodule=data_module, ckpt_path="best")

# Predict (only here as a demo and test)
# forecaster.to("cuda")
# xc, yc, xt, _ = test_samples
# yt_pred = forecaster.predict(xc, yc, xt)

if args.wandb:
    experiment.finish()